## feature engineering

In [1]:
# standard library imports
import csv
import datetime as dt
import json
import os
import statistics
import time

# third-party imports
import numpy as np
import pandas as pd
import requests

# google cloud storage
import gcsfs

import statsmodels.api as sm
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('steamspy_onehot.csv')

# number of languages
def cnt(var):
    return var.count(',')+1

# missing values considered as 1
res = []
for i in range(len(data.languages)):
    if pd.isna(data.languages[i]):
        tmp = 1
    else:
        tmp = cnt(data.languages[i])
    res.append(tmp)
    
data['n_lang'] = res

data['favorable_rate'] = data.positive/(data.positive+data.negative)

d = pd.DataFrame(data['developer'].value_counts().head(10)).reset_index()
p = pd.DataFrame(data['publisher'].value_counts().head(10)).reset_index()

data['top10_dev'] = [1 if i in d.index else 0 for i in data.developer]
data['top10_pub'] = [1 if i in p.index else 0 for i in data.publisher]

o = []
for i in data.owners:
    if i == '0 .. 20,000':
        j = 10000
    elif i == '20,000 .. 50,000':
        j = 35000
    elif i == '50,000 .. 100,000':
        j = 75000
    elif i == '100,000 .. 200,000':
        j = 150000
    elif i == '200,000 .. 500,000':
        j = 350000
    elif i == '500,000 .. 1,000,000':
        j = 750000
    elif i == '1,000,000 .. 2,000,000':
        j = 1500000
    elif i == '2,000,000 .. 5,000,000':
        j = 3500000
    elif i == '5,000,000 .. 10,000,000':
        j = 7500000
    elif i == '10,000,000 .. 20,000,000':
        j = 15000000
    elif i == '20,000,000 .. 50,000,000':
        j = 35000000
    elif i == '50,000,000 .. 100,000,000':
        j = 75000000
    else:
        j = 150000000
    o.append(j)
    
data['n_owner'] = o

data['if_dis'] = [1 if i != 0 else 0 for i in data.discount]


## lda

### preprocess

In [3]:
data.head()

,index,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,...,tag_World War II,tag_Wrestling,tag_Zombies,tag_e-sports,n_lang,favorable_rate,top10_dev,top10_pub,n_owner,if_dis
0,0,10,Counter-Strike,Valve,Valve,NaN,184083,4787,0,"10,000,000 .. 20,000,000",...,0.0,0.0,0.0,1170.0,8,0.974655,0,0,15000000,0
1,1,20,Team Fortress Classic,Valve,Valve,NaN,5227,871,0,"2,000,000 .. 5,000,000",...,0.0,0.0,0.0,0.0,9,0.857166,0,0,3500000,0
2,2,30,Day of Defeat,Valve,Valve,NaN,4869,543,0,"1,000,000 .. 2,000,000",...,246.0,0.0,0.0,0.0,5,0.899667,0,0,1500000,0
3,3,40,Deathmatch Classic,Valve,Valve,NaN,1790,400,0,"5,000,000 .. 10,000,000",...,0.0,0.0,0.0,0.0,9,0.817352,0,0,7500000,0
4,4,50,Half-Life: Opposing Force,Gearbox Software,Valve,NaN,12451,633,0,"5,000,000 .. 10,000,000",...,0.0,0.0,0.0,0.0,4,0.951620,0,0,7500000,0


In [4]:
data.shape

(29235, 525)

In [5]:
data['owners'][0].split(' .. ')

['10,000,000', '20,000,000']

In [6]:
data['owners_lbud'] = data['owners'].apply(lambda x: x.split(' .. ')[0])
data['owners_hbud'] = data['owners'].apply(lambda x: x.split(' .. ')[1])
int(data['owners_lbud'][0].replace(',', ''))

10000000

In [7]:
data['owners_lbud'] = data['owners_lbud'].apply(lambda x: int(x.replace(',', '')))
data['owners_hbud'] = data['owners_hbud'].apply(lambda x: int(x.replace(',', '')))

In [8]:
data['owners_mid'] = (data['owners_lbud'] + data['owners_hbud']) / 2
data.head()

,index,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,...,tag_e-sports,n_lang,favorable_rate,top10_dev,top10_pub,n_owner,if_dis,owners_lbud,owners_hbud,owners_mid
0,0,10,Counter-Strike,Valve,Valve,NaN,184083,4787,0,"10,000,000 .. 20,000,000",...,1170.0,8,0.974655,0,0,15000000,0,10000000,20000000,15000000.0
1,1,20,Team Fortress Classic,Valve,Valve,NaN,5227,871,0,"2,000,000 .. 5,000,000",...,0.0,9,0.857166,0,0,3500000,0,2000000,5000000,3500000.0
2,2,30,Day of Defeat,Valve,Valve,NaN,4869,543,0,"1,000,000 .. 2,000,000",...,0.0,5,0.899667,0,0,1500000,0,1000000,2000000,1500000.0
3,3,40,Deathmatch Classic,Valve,Valve,NaN,1790,400,0,"5,000,000 .. 10,000,000",...,0.0,9,0.817352,0,0,7500000,0,5000000,10000000,7500000.0
4,4,50,Half-Life: Opposing Force,Gearbox Software,Valve,NaN,12451,633,0,"5,000,000 .. 10,000,000",...,0.0,4,0.951620,0,0,7500000,0,5000000,10000000,7500000.0


In [9]:
data_filtered = data.drop(['index', 'developer', 'publisher', 'score_rank', 'positive', 'negative', \
                       'userscore', 'owners', 'languages', 'genre', 'tags'], axis = 1)

In [10]:
data_filtered[data_filtered.columns[pd.Series(data_filtered.columns).str.startswith('tag')]].sum().sort_values(ascending=False)[:10]

tag_Action          1253419.0
tag_Indie           1104360.0
tag_Adventure        888884.0
tag_Multiplayer      775060.0
tag_Singleplayer     749825.0
tag_Strategy         603831.0
tag_Casual           582095.0
tag_Free to Play     562104.0
tag_RPG              530886.0
tag_Simulation       501087.0
dtype: float64

In [11]:
data_filtered[data_filtered.columns[pd.Series(data_filtered.columns).str.startswith('genre')]].sum().sort_values(ascending=False)[:10]

genre_Indie          20504
genre_Action         12742
genre_Casual         10905
genre_Adventure      10666
genre_Strategy        5596
genre_Simulation      5521
genre_RPG             4624
genre_EarlyAccess     2478
genre_FreetoPlay      1914
genre_Sports          1419
dtype: int64

In [12]:
top_10tag = ["tag_Action", "tag_Indie", "tag_Adventure", "tag_Multiplayer", "tag_Singleplayer", \
             "tag_Strategy", "tag_Casual", "tag_Free to Play", "tag_RPG", "tag_Simulation"]

top_10genre = ["genre_Indie", "genre_Action", "genre_Casual", 'genre_Adventure', "genre_Strategy", \
              "genre_Simulation", "genre_RPG", "genre_EarlyAccess", 'genre_FreetoPlay', "genre_Sports"]

In [13]:
data_filtered = data_filtered.dropna(axis=0)
data_filtered = data_filtered.reset_index()
data_filtered.shape

(28841, 518)

In [14]:
data_tag_cluster = data_filtered.filter(regex='^tag',axis=1)
data_tag_cluster.shape
data_tag_cluster.shape

(28841, 428)

In [15]:
data_tag_cluster['owners_mid'] = data_filtered['owners_mid']
data_tag_cluster['favorable_rate'] = data_filtered['favorable_rate']
data_tag_cluster.head()

C:\Users\30283\AppData\Local\Temp/ipykernel_46840/3466670432.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_tag_cluster['owners_mid'] = data_filtered['owners_mid']
C:\Users\30283\AppData\Local\Temp/ipykernel_46840/3466670432.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_tag_cluster['favorable_rate'] = data_filtered['favorable_rate']


,"tag_""1990s""","tag_""Beat em up""","tag_""Shoot Em Up""",tag_1980s,tag_2.5D,tag_2D,tag_2D Fighter,tag_2D Platformer,tag_360 Video,tag_3D,...,tag_Werewolves,tag_Western,tag_Word Game,tag_World War I,tag_World War II,tag_Wrestling,tag_Zombies,tag_e-sports,owners_mid,favorable_rate
0,1177.0,0.0,0.0,256.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1170.0,15000000.0,0.974655
1,131.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3500000.0,0.857166
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,12.0,246.0,0.0,0.0,0.0,1500000.0,0.899667
3,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7500000.0,0.817352
4,132.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7500000.0,0.951620


In [16]:
train_tagclus, test_tagclus = train_test_split(data_tag_cluster, test_size=0.2, random_state=42, shuffle=True)
train_tagclus

,"tag_""1990s""","tag_""Beat em up""","tag_""Shoot Em Up""",tag_1980s,tag_2.5D,tag_2D,tag_2D Fighter,tag_2D Platformer,tag_360 Video,tag_3D,...,tag_Werewolves,tag_Western,tag_Word Game,tag_World War I,tag_World War II,tag_Wrestling,tag_Zombies,tag_e-sports,owners_mid,favorable_rate
4189,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,150000.0,0.640761
15662,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35000.0,0.796460
15208,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10000.0,0.800000
24447,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35000.0,0.557447
3739,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35000.0,0.667183
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21575,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10000.0,0.142857
5390,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10000.0,0.756757
860,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1500000.0,0.877563
15795,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,750000.0,0.856294


In [17]:
X_cluster = train_tagclus.drop(['owners_mid', 'favorable_rate'], axis = 1)
X_cluster

,"tag_""1990s""","tag_""Beat em up""","tag_""Shoot Em Up""",tag_1980s,tag_2.5D,tag_2D,tag_2D Fighter,tag_2D Platformer,tag_360 Video,tag_3D,...,tag_Web Publishing,tag_Well-Written,tag_Werewolves,tag_Western,tag_Word Game,tag_World War I,tag_World War II,tag_Wrestling,tag_Zombies,tag_e-sports
4189,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15662,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15208,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24447,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3739,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21575,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5390,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
860,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15795,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### lda model

In [18]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.


In [19]:
from gensim.models import LdaModel
from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary

In [20]:
#generating dictionary and corpos from trainning dataset
tag_lda = []
for i in X_cluster.index:
    temp = []
    for col in X_cluster.columns:
        if col != 'index':
            for j in range(int(X_cluster[col][i])):
                temp.append(col)
    tag_lda.append(temp)
    
common_dictionary = Dictionary(tag_lda)
common_corpus = [common_dictionary.doc2bow(text) for text in tag_lda]

In [21]:
# Set training parameters.
num_topics = 20
chunksize = 500 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 400
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = common_dictionary[0]  # This is only to "load" the dictionary.
id2word = common_dictionary.id2token

model = LdaModel(corpus=common_corpus, id2word=id2word, chunksize=chunksize, \
                   alpha='auto', eta='auto', \
                   iterations=iterations, num_topics=num_topics, \
                   passes=passes, eval_every=eval_every)

In [22]:
! pip install pyLDAvis

In [23]:
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [24]:
pyLDAvis.gensim_models.prepare(model, common_corpus, common_dictionary, sort_topics = False)

C:\Users\30283\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
0      0.181200 -0.000429       1        1  4.583724
1      0.003885  0.168172       2        1  4.593470
2     -0.159531  0.129015       3        1  4.297948
3     -0.260454  0.068213       4        1  7.421078
4     -0.289631  0.003563       5        1  9.527665
5      0.010776  0.144542       6        1  6.103418
6      0.140670  0.017303       7        1  2.997051
7      0.110521  0.067487       8        1  2.675661
8      0.138884  0.179434       9        1  2.792496
9     -0.158875 -0.182894      10        1  6.102378
10     0.098975  0.079174      11        1  3.918287
11     0.126480 -0.247489      12        1  2.308890
12    -0.115485  0.089059      13        1  2.367480
13     0.045920 -0.113638      14        1  4.707117
14     0.046157  0.008105      15        1  4.756044
15     0.131202  0.085297      16        1  6.871016
16    -0.078188 -0.162479      17        1  9.004779
17    -0.086040 -0.273503      18        1  5.736860
18     0.210166 -0.134128      19        1  6.171928
19    -0.096634  0.075196      20        1  3.062712, topic_info=                  Term          Freq         Total Category  logprob  loglift
8            tag_Indie  8.619080e+05  8.619080e+05  Default  30.0000  30.0000
22        tag_Strategy  5.082070e+05  5.082070e+05  Default  29.0000  29.0000
12             tag_RPG  4.349590e+05  4.349590e+05  Default  28.0000  28.0000
42      tag_Simulation  3.926270e+05  3.926270e+05  Default  27.0000  27.0000
1           tag_Action  1.058757e+06  1.058757e+06  Default  26.0000  26.0000
..                 ...           ...           ...      ...      ...      ...
379  tag_Battle Royale  6.142304e-02  1.054861e+00  Topic20 -16.2011   0.6425
342        tag_Sokoban  6.141806e-02  1.054782e+00  Topic20 -16.2012   0.6425
14    tag_Singleplayer  1.569182e+04  6.237584e+05  Topic20  -3.7503  -0.1968
84           tag_Co-op  4.940903e+03  3.791526e+05  Topic20  -4.9058  -0.8545
29     tag_Multiplayer  6.142014e-02  7.012017e+05  Topic20 -16.2012 -12.7647

[744 rows x 6 columns], token_table=      Topic      Freq               Term
term                                    
201      20  0.999940        tag_"1990s"
222      20  0.999950   tag_"Beat em up"
111       6  0.999974  tag_"Shoot Em Up"
156      20  0.999919          tag_1980s
296       6  0.999940           tag_2.5D
...     ...       ...                ...
240       3  0.999644    tag_World War I
241       3  0.999976   tag_World War II
225      20  0.999459      tag_Wrestling
124      12  0.999984        tag_Zombies
280       4  0.999984       tag_e-sports

[473 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20])

In [25]:
#print out topic distribution
for i,topic in model.show_topics(formatted=True, num_topics=20, num_words=10):
    print(str(i)+": "+ topic)
    print()

0: 0.139*"tag_Anime" + 0.106*"tag_Visual Novel" + 0.087*"tag_Nudity" + 0.068*"tag_Female Protagonist" + 0.068*"tag_Sexual Content" + 0.065*"tag_Choices Matter" + 0.061*"tag_Story Rich" + 0.046*"tag_Multiple Endings" + 0.044*"tag_Dating Sim" + 0.038*"tag_Mature"

1: 0.508*"tag_Strategy" + 0.070*"tag_RTS" + 0.045*"tag_Tactical" + 0.044*"tag_Tower Defense" + 0.044*"tag_Turn-Based Strategy" + 0.039*"tag_Turn-Based" + 0.033*"tag_Singleplayer" + 0.030*"tag_Turn-Based Tactics" + 0.026*"tag_Real Time Tactics" + 0.026*"tag_Multiplayer"

2: 0.157*"tag_Realistic" + 0.151*"tag_War" + 0.141*"tag_Tactical" + 0.122*"tag_Military" + 0.075*"tag_Vehicular Combat" + 0.070*"tag_Historical" + 0.050*"tag_World War II" + 0.047*"tag_Simulation" + 0.037*"tag_Naval Combat" + 0.032*"tag_Naval"

3: 0.204*"tag_FPS" + 0.178*"tag_Action" + 0.152*"tag_First-Person" + 0.145*"tag_Shooter" + 0.058*"tag_Difficult" + 0.051*"tag_Souls-like" + 0.048*"tag_e-sports" + 0.031*"tag_Fast-Paced" + 0.021*"tag_Lore-Rich" + 0.020*"ta

In [26]:
data_tag_cluster.head()

,"tag_""1990s""","tag_""Beat em up""","tag_""Shoot Em Up""",tag_1980s,tag_2.5D,tag_2D,tag_2D Fighter,tag_2D Platformer,tag_360 Video,tag_3D,...,tag_Werewolves,tag_Western,tag_Word Game,tag_World War I,tag_World War II,tag_Wrestling,tag_Zombies,tag_e-sports,owners_mid,favorable_rate
0,1177.0,0.0,0.0,256.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1170.0,15000000.0,0.974655
1,131.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3500000.0,0.857166
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,12.0,246.0,0.0,0.0,0.0,1500000.0,0.899667
3,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7500000.0,0.817352
4,132.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7500000.0,0.951620


In [27]:
data_tag_cluster.shape

(28841, 430)

In [28]:
## preprocessing entire data
tag_lda_total = []
for i in data_tag_cluster.index:
    temp = []
    for col in data_tag_cluster.columns:
        if col != 'index' and col != 'owners_mid' and col != 'favorable_rate':
            for j in range(int(data_tag_cluster[col][i])):
                temp.append(col)
    tag_lda_total.append(temp)
    
common_dictionary_total = Dictionary(tag_lda_total)
common_corpus_total = [common_dictionary.doc2bow(text) for text in tag_lda_total]

In [29]:
topic_decomposition = []
for i in range(len(common_corpus_total)):
    topic_decomposition.append(model[common_corpus_total[i]])

In [30]:
data_tag_cluster['topic_decomposition'] = topic_decomposition

C:\Users\30283\AppData\Local\Temp/ipykernel_46840/3606258918.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_tag_cluster['topic_decomposition'] = topic_decomposition


In [31]:
data_filtered['topic_decomposition'] = topic_decomposition
data_filtered['major_topic'] = data_filtered['topic_decomposition'].apply(lambda x: sorted(x, key=lambda tup: tup[1])[-1][0])
data_filtered['major_topic_percentage'] = data_filtered['topic_decomposition'].apply(lambda x: sorted(x, key=lambda tup: tup[1])[-1][1])

In [66]:
data_filtered.to_csv('gs://6893projectdata/steamspy_withlda.csv', index = False)

## sentiment analysis

In [33]:
df = pd.read_csv('steamspy_data.csv')

import ast
def gettag(var):
    if var == '[]':
        return 'NA'
    else:
        var = ast.literal_eval(var)
        return list(var)[0]
df['tag'] = df.tags.apply(gettag)

df.tag.value_counts()

Action               6023
Indie                4864
Adventure            3211
Casual               2698
Strategy             1959
                     ... 
Moddable                1
Basketball              1
Gun Customization       1
Motocross               1
Multiple Endings        1
Name: tag, Length: 319, dtype: int64

319 unqiue tags, making it impossible to conduct twitter analysis. 

## 1. popularity (dau)

In [34]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.decomposition import PCA
d1 = data.drop(['index','appid','name','developer','publisher','score_rank','owners','initialprice','discount','languages',
                'genre','tags'], axis=1)
d1 = d1.dropna()
x = d1.drop('average_forever', axis=1)
y = d1.average_forever

# standard scaler
scaler = preprocessing.StandardScaler()
xs = scaler.fit_transform(x)
# pca to reduce dimensionality
pca = PCA(n_components=10)
xp = pca.fit_transform(x)
# both
xsp = pca.fit_transform(xs)

# train_test_split， default: testsize=25%
xtrain, xtest, ytrain, ytest = train_test_split(x, y, random_state=1)
xtrains, xtests, ytrain, ytest = train_test_split(xs, y, random_state=1)
xtrainp, xtestp, ytrain, ytest = train_test_split(xp, y, random_state=1)
xtrainsp, xtestsp, ytrain, ytest = train_test_split(xsp, y, random_state=1)

In [35]:
import numpy as np
# Symmetric mean absolute percentage error: https://en.wikipedia.org/wiki/Symmetric_mean_absolute_percentage_error
def symmetric_mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs(y_true - y_pred) / (np.abs(y_true)+np.abs(y_pred)+0.01)) # add 0.01 to avoid nan

def lr(x,x1,y,y1):
    model = LinearRegression()
    model.fit(x,y)
    ypred = model.predict(x1)
    smape = symmetric_mean_absolute_percentage_error(y1, ypred)
    print(f'smape = {smape}')
    
# baseline
lr(xtrain, xtest, ytrain, ytest)
# scaling
lr(xtrains, xtests, ytrain, ytest)
# pca
lr(xtrainp, xtestp, ytrain, ytest)
# scaling + pca
lr(xtrainsp, xtestsp, ytrain, ytest)

smape = 0.731767516827416
smape = 0.7316189822899553
smape = 0.6980235060374226
smape = 0.7879116207947771


linear regression with pca has the lowest smape of ~70%.

In [36]:
from sklearn.ensemble import RandomForestRegressor

def rf(x,x1,y,y1):
    model = RandomForestRegressor()
    model.fit(x,y)
    ypred = model.predict(x1)
    smape = symmetric_mean_absolute_percentage_error(y1, ypred)
    print(f'smape = {smape}')
    
# baseline
rf(xtrain, xtest, ytrain, ytest)

smape = 0.04365220221067


In [37]:
# scaling isn't vital for tree model, so test pca in rf
rf(xtrainp, xtestp, ytrain, ytest)

smape = 0.3273320405099981


baseline random forest has the lowest smape of ~4%. 

In [38]:
# predict all games' dau using rf
model = RandomForestRegressor()
model.fit(xtrain,ytrain)
ypred_all = model.predict(x)

## 2. penetration (ownership)

In [39]:
x = d1.drop('n_owner', axis=1)
y = d1.n_owner

# standard scaler
scaler = preprocessing.StandardScaler()
xs = scaler.fit_transform(x)
# pca to reduce dimensionality
pca = PCA(n_components=10)
xp = pca.fit_transform(x)
# both
xsp = pca.fit_transform(xs)

# train_test_split， default: testsize=25%
xtrain, xtest, ytrain, ytest = train_test_split(x, y, random_state=1)
xtrains, xtests, ytrain, ytest = train_test_split(xs, y, random_state=1)
xtrainp, xtestp, ytrain, ytest = train_test_split(xp, y, random_state=1)
xtrainsp, xtestsp, ytrain, ytest = train_test_split(xsp, y, random_state=1)

In [40]:
# baseline
lr(xtrain, xtest, ytrain, ytest)
# scaling
lr(xtrains, xtests, ytrain, ytest)
# pca
lr(xtrainp, xtestp, ytrain, ytest)
# scaling + pca
lr(xtrainsp, xtestsp, ytrain, ytest)

smape = 2.936374991473809e-14
smape = 0.00019464528160158931
smape = 1.167191088132367e-07
smape = 0.668929559824371


In [41]:
rf(xtrain, xtest, ytrain, ytest)
rf(xtrainp, xtestp, ytrain, ytest)

smape = 5.7544810094476516e-05
smape = 2.4770289566104603e-05


In [42]:
from sklearn.ensemble import GradientBoostingRegressor
def gbr(x,x1,y,y1):
    model = GradientBoostingRegressor()
    model.fit(x,y)
    ypred = model.predict(x1)
    smape = symmetric_mean_absolute_percentage_error(y1, ypred)
    print(f'smape = {smape}')
    
gbr(xtrain, xtest, ytrain, ytest)
gbr(xtrainp, xtestp, ytrain, ytest)

smape = 0.00069944502710582
smape = 0.0006437883247959385


random forest shot the best performance for predicting ownership. 

In [43]:
model = RandomForestRegressor()
model.fit(xtrain,ytrain)
ypred_all_own = model.predict(x)

## 3. playability (ccu)

In [44]:
x = d1.drop('ccu', axis=1)
y = d1.ccu

# standard scaler
scaler = preprocessing.StandardScaler()
xs = scaler.fit_transform(x)
# pca to reduce dimensionality
pca = PCA(n_components=10)
xp = pca.fit_transform(x)
# both
xsp = pca.fit_transform(xs)

# train_test_split， default: testsize=25%
xtrain, xtest, ytrain, ytest = train_test_split(x, y, random_state=1)
xtrains, xtests, ytrain, ytest = train_test_split(xs, y, random_state=1)
xtrainp, xtestp, ytrain, ytest = train_test_split(xp, y, random_state=1)
xtrainsp, xtestsp, ytrain, ytest = train_test_split(xsp, y, random_state=1)

In [45]:
# baseline
lr(xtrain, xtest, ytrain, ytest)
# scaling
lr(xtrains, xtests, ytrain, ytest)
# pca
lr(xtrainp, xtestp, ytrain, ytest)
# scaling + pca
lr(xtrainsp, xtestsp, ytrain, ytest)

smape = 0.9536384602402759
smape = 0.9536196808498445
smape = 0.9709515368639502
smape = 0.9684083370031268


In [46]:
rf(xtrain, xtest, ytrain, ytest)
rf(xtrainp, xtestp, ytrain, ytest)

smape = 0.567861217710918
smape = 0.5894268360319722


In [47]:
gbr(xtrain, xtest, ytrain, ytest)
gbr(xtrainp, xtestp, ytrain, ytest)

smape = 0.8455005749932852
smape = 0.8373424207880649


In [48]:
model = RandomForestRegressor()
model.fit(xtrain,ytrain)
ypred_all_ccu = model.predict(x)

## 4. feedback (favorable_rate)

In [49]:
x = d1.drop('favorable_rate', axis=1)
y = d1.favorable_rate

# standard scaler
scaler = preprocessing.StandardScaler()
xs = scaler.fit_transform(x)
# pca to reduce dimensionality
pca = PCA(n_components=10)
xp = pca.fit_transform(x)
# both
xsp = pca.fit_transform(xs)

# train_test_split， default: testsize=25%
xtrain, xtest, ytrain, ytest = train_test_split(x, y, random_state=1)
xtrains, xtests, ytrain, ytest = train_test_split(xs, y, random_state=1)
xtrainp, xtestp, ytrain, ytest = train_test_split(xp, y, random_state=1)
xtrainsp, xtestsp, ytrain, ytest = train_test_split(xsp, y, random_state=1)

In [50]:
# baseline
lr(xtrain, xtest, ytrain, ytest)
# scaling
lr(xtrains, xtests, ytrain, ytest)
# pca
lr(xtrainp, xtestp, ytrain, ytest)
# scaling + pca
lr(xtrainsp, xtestsp, ytrain, ytest)

smape = 0.13686071385411744
smape = 0.13699463900634604
smape = 0.14033458097132778
smape = 0.13982610224409925


In [51]:
rf(xtrain, xtest, ytrain, ytest)
rf(xtrainp, xtestp, ytrain, ytest)

smape = 0.0020519952019784926
smape = 0.01595076901548866


In [52]:
gbr(xtrain, xtest, ytrain, ytest)
gbr(xtrainp, xtestp, ytrain, ytest)

smape = 0.020283921152559715
smape = 0.05614107376364018


In [53]:
model = RandomForestRegressor()
model.fit(xtrain,ytrain)
ypred_all_fr = model.predict(x)

## generate csv

In [54]:
d1['i'] = d1.index
data['i'] = data.index
m = data[['appid', 'name', 'publisher','i']]
r = m.merge(d1, on='i', how='inner')

In [55]:
l = ['appid', 'name', 'publisher'] + [col for col in d1.columns if 'genre' in col] 
target = r[l]
target['popularity'] = ypred_all
target['penetration'] = ypred_all_own
target['playability'] = ypred_all_ccu
target['feedback'] = ypred_all_fr

C:\Users\30283\AppData\Local\Temp/ipykernel_46840/2893821350.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target['popularity'] = ypred_all
C:\Users\30283\AppData\Local\Temp/ipykernel_46840/2893821350.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target['penetration'] = ypred_all_own
C:\Users\30283\AppData\Local\Temp/ipykernel_46840/2893821350.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


In [56]:
def score_pop(var):
    return 25*((var-target.popularity.min())/(target.popularity.max()-target.popularity.min()))

def score_pen(var):
    return 25*((var-target.penetration.min())/(target.penetration.max()-target.penetration.min()))

def score_pla(var):
    return 25*((var-target.playability.min())/(target.playability.max()-target.playability.min()))

def score_fee(var):
    return 25*((var-target.feedback.min())/(target.feedback.max()-target.feedback.min()))

target['score'] = target.popularity.apply(score_pop) + target.penetration.apply(score_pen) + target.playability.apply(score_pla) + target.feedback.apply(score_fee)

C:\Users\30283\AppData\Local\Temp/ipykernel_46840/151741785.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target['score'] = target.popularity.apply(score_pop) + target.penetration.apply(score_pen) + target.playability.apply(score_pla) + target.feedback.apply(score_fee)


In [57]:
target.sort_values('score', ascending=False)

,appid,name,publisher,genre_Accounting,genre_Action,genre_Adventure,genre_Animation&Modeling,genre_AudioProduction,genre_Casual,genre_Design&Illustration,...,genre_Utilities,genre_VideoProduction,genre_Violent,genre_WebPublishing,genre_nan,popularity,penetration,playability,feedback,score
22,570,Dota 2,Valve,0,1,0,0,0,0,0,...,0,0,0,0,0,26482.50,118850000.0,425257.64,0.838998,79.272841
25,730,Counter-Strike: Global Offensive,Valve,0,1,0,0,0,0,0,...,0,0,0,0,0,25673.86,74250000.0,559763.93,0.841292,75.518299
13755,578080,PUBG: BATTLEGROUNDS,"KRAFTON, Inc.",0,1,1,0,0,0,0,...,0,0,0,0,0,22909.24,63600000.0,230266.95,0.698418,53.496776
19,440,Team Fortress 2,Valve,0,1,0,0,0,0,0,...,0,0,0,0,0,10645.74,54950000.0,253652.75,0.855469,50.023391
2780,271590,Grand Theft Auto V,Rockstar Games,0,1,1,0,0,0,0,...,0,0,0,0,0,14975.70,44150000.0,95451.75,0.875029,43.513848
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16050,650310,Journey of the Sword,Cipher Hive,0,0,0,0,0,0,0,...,0,0,0,0,0,0.00,10000.0,0.00,0.000000,0.000000
16051,650340,TublerVR,SiliconWetware,0,1,1,0,0,1,0,...,0,0,0,0,0,0.00,10000.0,0.00,0.000000,0.000000
10497,495540,Dashy Square VR,Kastriot Sulejmani,0,1,0,0,0,0,0,...,0,0,0,0,0,0.00,10000.0,0.00,0.000000,0.000000
10510,495770,Free Balling,Skunk Software,0,1,0,0,0,1,0,...,0,0,0,0,0,0.00,10000.0,0.00,0.000000,0.000000


In [64]:
target.to_csv('gs://6893projectdata/game_scorecard_lda.csv', index=False)

## generate csv for web interface

In [59]:
def movecol(df, cols_to_move=[], ref_col='', place='After'):
    
    cols = df.columns.tolist()
    if place == 'After':
        seg1 = cols[:list(cols).index(ref_col) + 1]
        seg2 = cols_to_move
    if place == 'Before':
        seg1 = cols[:list(cols).index(ref_col)]
        seg2 = cols_to_move + [ref_col]
    
    seg1 = [i for i in seg1 if i not in seg2]
    seg3 = [i for i in cols if i not in seg1 + seg2]
    
    return(df[seg1 + seg2 + seg3])

In [60]:
web = target.drop(['popularity','penetration','playability','feedback'], axis=1)
web = movecol(web, 
             cols_to_move=['score'], 
             ref_col='publisher',
             place='After')

In [61]:
web.iloc[:,2] = web.iloc[:,2].fillna('NA')

In [65]:
web.to_csv('gs://6893projectdata/webdata.csv', index = False)